In [1]:
#@title 查看库版本/更新库 { run: "auto", vertical-output: true, form-width: "500px" }

import os 
import sys

print("sys.version :" , sys.version)
print("sys.executable :" , sys.executable)

count = 3   #最大检测次数，即第一次检测不存在则安装，会有安装失败的情况，就会再来一次

while count:
  try:
    import tushare as ts
    import tensorflow as tf
    import matplotlib as mpt
    import numpy as np
    import pandas as pd

    # import pydrive 
    import tushare as ts

    import seaborn as sns
    import matplotlib.pyplot as plt

    #显示版本

    print("tensorflow :" , tf.__version__)
    print("matplotlib :" , mpt.__version__)
    print("seaborn :" , sns.__version__)
    print("numpy :" , np.__version__)
    print("pandas :" , pd.__version__)
    # print("pydrive :" , pydrive.__version__)

    print("tushare :" , ts.__version__)
    
    break

  except:

    print('模块未安装,现在准备开始安装 ... ')

    # os.system('pip install urllib2')
    # os.system(' pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tensorflow  ')

    !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple numpy 
    !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pandas 
    !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple matplotlib 
    !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple seaborn 
    !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tushare 


    # !pip install backtrader 
    # !pip install yagmail 
    # !pip install keyring 
    # !pip install sklearn 
    # !pip install -U -q PyDrive 
    # !pip install factor-analyzer

    count -= 1
    continue

sys.version : 3.10.4 (main, Nov 18 2022, 01:20:00) [GCC 9.4.0]
sys.executable : /home/codespace/.python/current/bin/python3


2022-12-18 08:28:57.845386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 08:29:01.051541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-18 08:29:01.051584: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-18 08:29:09.221513: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

tensorflow : 2.11.0
matplotlib : 3.6.2
seaborn : 0.12.1
numpy : 1.23.5
pandas : 1.5.1
tushare : 1.2.89


In [4]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import tushare as ts
import pandas as pd
import numpy as np
class MAStrategy(object):
    def __init__(self,bars,short_window=5,long_window=60):
        self.bars=bars
        self.short_window=short_window
        self.long_window=long_window

    def MA(self,price,windows=5,period=1):
        MAprice=pd.Series(price).rolling(window=windows).mean()
        return MAprice

    def gen_signal(self):
        signals=pd.DataFrame(index=self.bars.index)
        signals['flag']=0
        signals['sma']=self.MA(self.bars['close'],self.short_window,1)
        signals['lma']=self.MA(self.bars['close'],self.long_window,1)
        signals.loc[self.short_window:,['flag']]=np.where(signals['sma'][self.short_window:]>signals['lma'][self.short_window:],1,0)
        signals['signal']=signals['flag'].diff().fillna(signals['flag'][0])
        return signals

class MATrade(object):
    def __init__(self,bars,signals,init_capital=100000):
        self.bar=bars
        self.init_capital=init_capital
        self.signals=signals
    def gen_position(self):
        positions=self.signals['flag']*1000
        return positions
    def trade_positions(self):
        positions=self.signals['signal']*1000
        return positions
    def trade_tracing(self):
        capital=pd.DataFrame(index=self.signals.index)
        capital['hold']=self.gen_position()*self.bar['close']#持仓
        capital['rest']=self.init_capital-(self.trade_positions()*bars['close']).cumsum()#用总资金减去每天的市值得出剩余资金
        capital['total']=capital['hold']+capital['rest']#总资产变化
        capital['return']=capital['total'].pct_change().fillna(capital['total'][0]/self.init_capital-1)
        return capital
if __name__=='__main__':
    bars=ts.get_hist_data('002211',start='2020-01-01',end='2021-03-01').sort_index()
    test_strategy=MAStrategy(bars)
    signals=test_strategy.gen_signal()
    test_trade=MATrade(bars,signals)
    capital=test_trade.trade_tracing()
    print(capital,signals)


本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2
              hold      rest     total    return
date                                            
2020-01-02     0.0  100000.0  100000.0  0.000000
2020-01-03     0.0  100000.0  100000.0  0.000000
2020-01-06     0.0  100000.0  100000.0  0.000000
2020-01-07     0.0  100000.0  100000.0  0.000000
2020-01-08     0.0  100000.0  100000.0  0.000000
...            ...       ...       ...       ...
2021-02-23     0.0   99870.0   99870.0  0.000000
2021-02-24     0.0   99870.0   99870.0  0.000000
2021-02-25  6020.0   93850.0   99870.0  0.000000
2021-02-26  5630.0   93850.0   99480.0 -0.003905
2021-03-01  5920.0   93850.0   99770.0  0.002915

[279 rows x 4 columns]             flag    sma       lma  signal
date                                     
2020-01-02     0    NaN       NaN     0.0
2020-01-03     0    NaN       NaN     0.0
2020-01-06     0    NaN       NaN     0.0
2020-01-07     0    NaN       NaN     0.0
2020-01-08     0  6.460       NaN  

In [2]:
from decimal import *

s = "123.4560"
d = Decimal(s)

print(s, type(s), d , type(d), d+ 1000)


123.4560 <class 'str'> 123.4560 <class 'decimal.Decimal'> 1123.4560
